In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescentPyTorch, CarliniLInfMethod, CarliniL2Method
from art.estimators.classification import PyTorchClassifier
import sys
sys.path.append('../architectures/')
sys.path.append('../adversarial/')
from wideresnet import WideResNet

In [2]:
# sd = torch.load('BEST_model-nn-epoch74-robacc57.pt')
# model = WideResNet(depth=34)
# model = nn.DataParallel(model)
# model.load_state_dict(sd)
# torch.save(model.module.state_dict(), 'best_atent_model_74.pt')

In [4]:
device = torch.device('cuda')

In [13]:
model = WideResNet(depth=34)
#model.load_state_dict(torch.load('model-nn-atent-76-robacc57.pt'))
model.load_state_dict(torch.load('model_cifar_wrn.pt'))
#model.load_state_dict(torch.load('best_atent_model_74.pt'))
model.eval();

In [14]:
loss = nn.CrossEntropyLoss(reduction='mean')
optim = torch.optim.SGD(params = model.parameters(), lr=0.01)
model = model.to(device)
pytorch_clf = PyTorchClassifier(model, loss=loss, optimizer=optim, input_shape=(1,32,32), nb_classes=10, device_type='gpu', clip_values=(0,1), preprocessing=None)

In [15]:
ds = CIFAR10('../../data/CIFAR10/', download=True, train=False, transform=Compose([ToTensor()]))
dl = DataLoader(ds, batch_size=128)

Files already downloaded and verified


In [8]:
correct = 0.0
correct_2 = 0.0
for x, y in dl:
    x = x.to(device)
    #y = y.to(device)
    #preds = torch.argmax(torch.tensor(pytorch_clf.predict(x.cpu().detach())), dim=-1)
    #correct+=preds.eq(y.view_as(preds)).sum().item()
    preds2 = torch.argmax(torch.tensor(model(x)), axis=-1).cpu()
    correct_2+=preds2.eq(y.view_as(preds2)).sum().item()
    #correct += (preds==y).sum()
#print(correct/10000.0)
print(correct_2/10000.0)

/data/ameya/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


0.8521


In [16]:
attack = ProjectedGradientDescentPyTorch(pytorch_clf, norm=np.inf, eps=0.031, eps_step=0.007, num_random_init=10, max_iter=20, batch_size=128)
#attack = CarliniLInfMethod(pytorch_clf, learning_rate=0.01, max_iter=10,  eps=0.031, batch_size=128)

In [17]:
correct = 0.0
correct_2 = 0.0
epsilons = []
for x, y in dl:
    x = x.to(device)
    #y = y.to(device)
    preds = torch.argmax(torch.tensor(pytorch_clf.predict(x.cpu())), dim=-1)
    x_adv = attack.generate(x.cpu())
    preds2 = torch.argmax(torch.tensor(pytorch_clf.predict(x_adv)), dim=-1)
    tmp_eps = torch.norm(torch.tensor(x_adv).view(-1, 32*32*3).cpu() - x.view(-1, 32*32*3).cpu(), dim=1, p=np.inf)
    epsilons.extend(tmp_eps.tolist())
    correct+=preds.eq(y.view_as(preds)).sum().item()
    correct_2 += preds2.eq(y.view_as(preds)).sum().item()
    #correct += (preds==y).sum()
print(correct/10000.0)
print(correct_2/10000.0)

0.8492
0.6307


In [18]:
np.save('trades_eps_pgd', epsilons)

In [13]:
epsilons

[0.02057778835296631,
 1.1754943508222875e-38,
 0.03100001811981201,
 1.1754943508222875e-38,
 0.03100001811981201,
 1.1754943508222875e-38,
 0.015582144260406494,
 0.026858940720558167,
 0.030999988317489624,
 0.02630174160003662,
 0.030997902154922485,
 1.1754943508222875e-38,
 0.02907806634902954,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 0.03100001811981201,
 1.1754943508222875e-38,
 0.030982643365859985,
 1.1754943508222875e-38,
 0.030991166830062866,
 1.1754943508222875e-38,
 0.030321985483169556,
 1.1754943508222875e-38,
 0.030994534492492676,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 0.03099769353866577,
 0.03100001811981201,
 0.029755502939224243,
 1.1754943508222875e-38,
 0.03044813871383667,
 1.1754943508222875e-38,
 0.00635296106338501,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38,
 1.1754943508222875e-38

In [ ]:
model2 = WideResNet(depth=34, num_classes=10)
model2 = model2.to(device)
model2 = nn.DataParallel(model2)
model2.load_state_dict(torch.load('model-nn-epoch76-robacc57.pt'))
model2.eval();

In [14]:
correct_2 = 0.0
for idx, (x, y) in enumerate(dl):
    print(idx)
    x = x.to(device)
    y = y.to(device)
#     preds = torch.argmax(torch.tensor(pytorch_clf.predict(x)), axis=-1)
#     correct+=preds.eq(y.view_as(preds)).sum().item()
    preds2 = torch.argmax(model2(x), axis=-1)
    correct_2+=preds2.eq(y.view_as(preds2)).sum().item()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


In [15]:
correct_2

8567.0